In [20]:
if COLAB:
    import os
    import subprocess
    def execute_and_get_output(command):
    try:
        result = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        print(result.decode('utf-8'))
    except subprocess.CalledProcessError as e:
        raise RuntimeError(
            "command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output.decode('utf-8')))
    def mount_and_pull(branch_name, drive, os):
        mgc = get_ipython().magic
        execute_and_get_output('git fetch')
        execute_and_get_output('git add -A')
        execute_and_get_output('git reset --hard')
        execute_and_get_output('git checkout {}'.format(branch_name))
        execute_and_get_output('git pull')
        execute_and_get_output('git branch')
        mgc('cd project_road_segmentation')
        execute_and_get_output('ls')
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/Shareddrives/ML_Road_Segmentation/CS-433-project-2'
    os.chdir(drive_path)
    BRANCH_NAME = 'ben-crap'
    mount_and_pull(BRANCH_NAME, drive, os)

In [16]:
import os
import sys
import gzip
import code
import urllib
import numpy as np
from tqdm import tqdm
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [17]:
%reload_ext autoreload
%autoreload 2

In [18]:
sys.path.append('./')
from helpers.helpers import *
from helpers.mask_to_submission import *

In [19]:
COLAB = False
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
GENERATE_PREDICTION = False #If True, will generate a CSV to submit on AICrowd

PREDICTIONS_SAVE_DIR = 'predictions/'
MODELS_SAVE_DIR = 'model_save/'
MODEL_NAME = 'new-u-net' #Chose between cnn_handmade and u-net

TRAINING_SAMPLES = 100 #max 100

TRAINING_SIZE = 80 # Size of the training set in percentage, integer between 0 and 100, the remaining part is for testing
VALIDATION_SIZE = 0.20  # Size of the validation set, float between 0 and 1
SEED = 66478  # Set to None for random seed.
NUM_EPOCHS = 15

NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
BATCH_SIZE = 16  # 64
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16
IMAGE_SIZE = 400
if IMAGE_SIZE % IMG_PATCH_SIZE != 0 :
    print('Warning : the image size is not a multiple of the patch size')

In [28]:
from models.cnn_tf import *
from models.unet_tf import *
from models.other_unet_tf import *

models_dict = {
    'cnn_handmade' : {
        'name' : 'cnn',
        'model' : CNN,
        'save_dir' : MODELS_SAVE_DIR + 'cnn_handmade/'
    },
    'u-net' : {
        'name' : 'u-net',
        'model' : UNet,
        'save_dir' : MODELS_SAVE_DIR + 'u-net/'
    },
    'new-u-net' : {
        'name' : 'new-u-net',
        'model' : get_unet,
        'save_dir' : MODELS_SAVE_DIR + 'new-u-net/'
    },
}

model_chosen = models_dict[MODEL_NAME]

In [22]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

def extract_data(filename, num_images):
    imgs = []
    for i in tqdm(range(1, num_images+1)):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print('File ' + image_filename + ' does not exist')
            
    print('Loaded {} training images'.format(len(imgs)))
    return np.asarray(imgs)

images = extract_data(train_data_filename, TRAINING_SAMPLES)
groundtruths = extract_data(train_labels_filename, TRAINING_SAMPLES)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Loaded 100 training images


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 873.38it/s]

Loaded 100 training images


In [23]:
X_train, X_test, y_train, y_test = train_test_split(images, groundtruths,\
                                                    train_size= TRAINING_SIZE/100, random_state=SEED)

In [24]:
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.keras import layers, models

#This cell is required for me (Ben), otherwise I get "convolution algorithm not found"
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("Set to true")
    print(physical_devices)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Set to true
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [25]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [29]:
if RESTORE_MODEL:
    # It can be used to reconstruct the model identically.
    model = models.load_model(model_chosen['save_dir'])
else:  
    model = model_chosen['model']()
    model.compile(optimizer='adam',
            loss="binary_crossentropy",
            metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs = NUM_EPOCHS ,validation_split=VALIDATION_SIZE)

Epoch 1/15


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_1/conv2d_11/Conv2D (defined at <ipython-input-29-78ac3239c2a8>:9) ]] [Op:__inference_train_function_5742]

Function call stack:
train_function


In [ ]:
if not RESTORE_MODEL:
    model.save(model_chosen['save_dir'])

In [ ]:
with tf.device('/cpu:0'):
    pred_train_original = model.predict(X_train).squeeze()

In [ ]:
pred_train = pred_train_original.copy()
print(pred_train[0].max(),' prediction ', pred_train[0].min())
print(y_train[0].max(),' label (groundtruth) ', y_train[0].min())
print(X_train[0].max(),' training image ', X_train[0].min())

THRESHOLD = ((pred_train.max() + pred_train.min()) / 2)
pred_train[pred_train >= THRESHOLD] = 1.0
pred_train[pred_train < THRESHOLD] = 0.0
pred_train = (pred_train -1)*-1 #Inverting the labels to match the images
print(pred_train[0])
for i in np.random.randint(int(TRAINING_SAMPLES*TRAINING_SIZE/100), size=4):
    plt.figure()
    img1 = concatenate_images(X_train[i], pred_train_original[i])
    img2 = concatenate_images(img1, pred_train[i])
    img3 = concatenate_images(img2, y_train[i])
    plt.imshow(img3)

In [ ]:
if COLAB:
    from helpers.colab import download_model
    from google.colab import files
    download_model(MODEL_NAME, model_chosen['save_dir'], files)